In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import requests
import pandas as pd
import json


In [3]:
# Substitua 'SUA_CHAVE_DE_API' pela sua chave de API
API_KEY = 'RGAPI-f996c09d-3987-4e3b-aaed-0fa0469d0c3b'

# Região do servidor (por exemplo, 'br1' para o servidor brasileiro)
REGION = 'br1'

# Nome de invocador (seu nome de jogador)
SUMMONER_NAME = 'teteco25'

# URL base da API da Riot Games
BASE_URL = f'https://{REGION}.api.riotgames.com/lol'


In [ ]:
listDivision = ['I','II','III','IV']
listTier = ['DIAMOND','EMERALD','PLATINUM','GOLD','SILVER','BRONZE','IRON']
#RANKED_SOLO_5x5

In [4]:
def filter_data(data):
    filtered_data = []
    for item in data:
        localized_names = item["localizedNames"].get("pt_BR")
        if localized_names:
            item["localizedNames"] = localized_names
            filtered_data.append(item)
    return filtered_data


In [5]:
def obter_id_invocador():
    summoner_url = f'https://{REGION}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{SUMMONER_NAME}'
    headers = {'X-Riot-Token': API_KEY}

    response = requests.get(summoner_url, headers=headers)

    if response.status_code == 200:
        match_history_data = response.json()
        return match_history_data['puuid']  # Retorna o ID do invocador
    else:
        print(f'Erro ao obter ID do invocador. Código de status: {response.status_code}')
        return None


In [6]:
def getConfigChallenge():
    summoner_url = f'https://{REGION}.api.riotgames.com/lol/challenges/v1/challenges/config'
    headers = {'X-Riot-Token': API_KEY}

    response = requests.get(summoner_url, headers=headers)

    if response.status_code == 200:
        filtered_data = filter_data(response.json())
        return filtered_data
    else:
        print(f'Erro ao obter ID do invocador. Código de status: {response.status_code}')
        return None

In [7]:
def getChallengeById(idPartida):
    match_history_url = f'https://{REGION}.api.riotgames.com/lol/challenges/v1/player-data/{idPartida}'
    headers = {'X-Riot-Token': API_KEY}

    response = requests.get(match_history_url, headers=headers)

    if response.status_code == 200:
        player_challenges_data = response.json()
        return player_challenges_data
    else:
        print(f'Erro ao obter histórico de partidas. Código de stasstus: {response.status_code}')
        return None

In [26]:
if __name__ == '__main__':
    # Obter o ID do invocador
    account_id = obter_id_invocador()

    if account_id:
        # Obter os dados dos desafios de configuração
        dfConfig = pd.DataFrame(getConfigChallenge())

        # Obter os dados do desafio do usuário
        dfUserChallenge = getChallengeById(account_id)
        
        df = pd.DataFrame(dfUserChallenge['challenges'])
        
        dfConfig['name'] = dfConfig['localizedNames'].apply(lambda x: x.get('name', ''))
        dfConfig['description'] = dfConfig['localizedNames'].apply(lambda x: x.get('description', ''))
        dfConfig['shortDescription'] = dfConfig['localizedNames'].apply(lambda x: x.get('shortDescription', ''))
        
        # Fundir os DataFrames df e dfConfig com base na correspondência entre df['challengeId'] e dfConfig['id']
        df_merged = pd.merge(df, dfConfig[['id', 'name','shortDescription']], left_on='challengeId', right_on='id', how='left')
        df_merged = df_merged.drop(columns=['id'])
        # Iterar sobre os desafios do usuário e imprimir o nome do desafio
        for index, row in df_merged.iterrows():
            print(f"ID: {row['challengeId']}")
            print(f"ID: {row['level']}")
            print(f"ID: {row['name']}")
            print("=" * 50)
    else:
        print("Nenhum histórico de partidas encontrado.")

ID: 0
ID: PLATINUM
ID: CRISTAL
ID: 1
ID: GOLD
ID: IMAGINAÇÃO
ID: 2
ID: PLATINUM
ID: ESPECIALIZAÇÃO
ID: 3
ID: PLATINUM
ID: VIVÊNCIA
ID: 4
ID: PLATINUM
ID: TRABALHO EM EQUIPE
ID: 5
ID: DIAMOND
ID: COLEÇÃO
ID: 302100
ID: DIAMOND
ID: Artimanha
ID: 302101
ID: MASTER
ID: Festinha no Nível 1
ID: 302102
ID: IRON
ID: Farme Campeões, Não Acampamentos
ID: 302103
ID: MASTER
ID: Presença Devastadora
ID: 302104
ID: MASTER
ID: Rolezinho Nas Rotas
ID: 302105
ID: GRANDMASTER
ID: Sistema Global de Defesa
ID: 302106
ID: DIAMOND
ID: Power Play do Barão
ID: 203303
ID: DIAMOND
ID: Sem Clemência
ID: 203302
ID: PLATINUM
ID: Não Quero Nem Saber
ID: 203301
ID: PLATINUM
ID: Especialista em Abates
ID: 203300
ID: PLATINUM
ID: Dizimador
ID: 301100
ID: PLATINUM
ID: Caçador de Monstros
ID: 301101
ID: BRONZE
ID: Pontualidade Frequente
ID: 203305
ID: PLATINUM
ID: Num Piscar de Olhos
ID: 301102
ID: DIAMOND
ID: Pegou, Levou
ID: 203304
ID: GOLD
ID: Investida Dividida Bem-Sucedida
ID: 510006
ID: MASTER
ID: Digno da Realeza

In [27]:
df_merged


,challengeId,percentile,level,value,achievedTime,position,playersInLevel,name,shortDescription
0,0,0.067,PLATINUM,9595.0,1.688062e+12,NaN,NaN,CRISTAL,Regras especiais para o Cristal
1,1,0.146,GOLD,1130.0,1.666183e+12,NaN,NaN,IMAGINAÇÃO,Divisão IMAGINAÇÃO
2,2,0.092,PLATINUM,2045.0,1.687872e+12,NaN,NaN,ESPECIALIZAÇÃO,Divisão ESPECIALIZAÇÃO
3,3,0.097,PLATINUM,2100.0,1.675272e+12,NaN,NaN,VIVÊNCIA,Divisão VIVÊNCIA
4,4,0.064,PLATINUM,2295.0,1.688429e+12,NaN,NaN,TRABALHO EM EQUIPE,Divisão TRABALHO EM EQUIPE
...,...,...,...,...,...,...,...,...,...
299,402403,0.056,DIAMOND,1229.0,1.688563e+12,NaN,NaN,Adeus à Escuridão,Posicione Sentinelas de Controle
300,402400,0.077,DIAMOND,540.0,1.687973e+12,NaN,NaN,Engenhosidade,Ganhe pontos nos desafios da categoria Engenho...
301,402401,0.089,DIAMOND,1550.0,1.690223e+12,NaN,NaN,Escuridão Absoluta,Elimine sentinelas
302,402408,0.069,DIAMOND,197300.0,1.691349e+12,NaN,NaN,Todo Mundo Paga!,Ganhe recompensas em ouro


,id,localizedNames,state,leaderboard,thresholds,endTimestamp,name,description,shortDescription
0,0,"{'description': '', 'name': 'CRISTAL', 'shortD...",ENABLED,True,"{'DIAMOND': 13800.0, 'BRONZE': 750.0, 'GOLD': ...",NaN,CRISTAL,,Regras especiais para o Cristal
1,1,"{'description': '', 'name': 'IMAGINAÇÃO', 'sho...",ENABLED,False,"{'DIAMOND': 2000.0, 'BRONZE': 175.0, 'GOLD': 7...",NaN,IMAGINAÇÃO,,Divisão IMAGINAÇÃO
2,2,"{'description': '', 'name': 'ESPECIALIZAÇÃO', ...",ENABLED,False,"{'DIAMOND': 3200.0, 'BRONZE': 275.0, 'GOLD': 1...",NaN,ESPECIALIZAÇÃO,,Divisão ESPECIALIZAÇÃO
3,3,"{'description': '', 'name': 'VIVÊNCIA', 'short...",ENABLED,False,"{'DIAMOND': 2700.0, 'BRONZE': 275.0, 'GOLD': 1...",NaN,VIVÊNCIA,,Divisão VIVÊNCIA
4,4,"{'description': '', 'name': 'TRABALHO EM EQUIP...",ENABLED,False,"{'DIAMOND': 3800.0, 'BRONZE': 200.0, 'GOLD': 1...",NaN,TRABALHO EM EQUIPE,,Divisão TRABALHO EM EQUIPE
...,...,...,...,...,...,...,...,...,...
357,402403,{'description': 'Posicione Sentinelas de Contr...,ENABLED,True,"{'DIAMOND': 900.0, 'BRONZE': 40.0, 'GOLD': 200...",NaN,Adeus à Escuridão,Posicione Sentinelas de Controle úteis. Uma se...,Posicione Sentinelas de Controle
358,402400,{'description': 'Ganhe pontos nos desafios da ...,ENABLED,True,"{'DIAMOND': 380.0, 'BRONZE': 40.0, 'GOLD': 135...",NaN,Engenhosidade,Ganhe pontos nos desafios da categoria Engenho...,Ganhe pontos nos desafios da categoria Engenho...
359,402401,"{'description': 'Elimine sentinelas', 'name': ...",ENABLED,True,"{'DIAMOND': 1200.0, 'BRONZE': 50.0, 'GOLD': 20...",NaN,Escuridão Absoluta,Elimine sentinelas,Elimine sentinelas
360,402408,{'description': 'Consiga recompensas em ouro p...,ENABLED,True,"{'DIAMOND': 160000.0, 'BRONZE': 5000.0, 'GOLD'...",NaN,Todo Mundo Paga!,Consiga recompensas em ouro por neutralizar Ca...,Ganhe recompensas em ouro


0      {'description': '', 'name': 'CRISTAL', 'shortD...
1      {'description': '', 'name': 'IMAGINAÇÃO', 'sho...
2      {'description': '', 'name': 'ESPECIALIZAÇÃO', ...
3      {'description': '', 'name': 'VIVÊNCIA', 'short...
4      {'description': '', 'name': 'TRABALHO EM EQUIP...
                             ...                        
357    {'description': 'Posicione Sentinelas de Contr...
358    {'description': 'Ganhe pontos nos desafios da ...
359    {'description': 'Elimine sentinelas', 'name': ...
360    {'description': 'Consiga recompensas em ouro p...
361    {'description': 'Ganhe ouro', 'name': 'Receber...
Name: localizedNames, Length: 362, dtype: object